# Singular Value Decomposition
Author: Pierre Nugues

Principal component analysis on characters of Salammbô. French and English chapters

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler, Normalizer
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from statistics import mean, stdev

## Utilities

### We build the indices

In [ ]:
def merge_dicts(dict_list):
    """
    Merges a list of dictionaries
    :param dict_list:
    :return:
    """
    dict_collector = {}
    for dict in dict_list:
        for key in dict:
            if key in dict_collector:
                dict_collector[key] += dict[key]
            else:
                dict_collector[key] = dict[key]
    return dict_collector


def build_idx_from_list(documents):
    """
    Builds indexes for the documents (rows)
    Build reverse index
    :param documents:
    :param chars:
    :return:
    """
    rev_doc_idx = {idx: file for idx, file in enumerate(documents)}
    doc_idx = {value: key for key, value in rev_doc_idx.items()}
    return doc_idx, rev_doc_idx


def build_idx_from_dict(chars):
    """
    Builds indexes for the characters (columns)
    Build reverse index
    :param documents:
    :param chars:
    :return:
    """
    rev_char_idx = {idx: char for idx, char in enumerate(sorted(chars.keys()))}
    char_idx = {value: key for key, value in rev_char_idx.items()}
    return char_idx, rev_char_idx

### We count the letters

In [ ]:
def count_letters(text, lc=True):
    letter_count = {}
    if lc:
        text = text.lower()
    for letter in text:
        if letter.lower().isalpha():
            if letter in letter_count:
                letter_count[letter] += 1
            else:
                letter_count[letter] = 1
    return letter_count

### We build the matrices

In [ ]:
def build_matrix(doc_idx, rev_doc_idx, char_idx, rev_char_idx, counts_by_chapter):
    X = np.zeros((len(doc_idx.keys()), len(char_idx.keys())))
    for i in rev_doc_idx.keys():
        for j in rev_char_idx.keys():
            if rev_char_idx[j] in counts_by_chapter[i]:
                X[i, j] = counts_by_chapter[i][rev_char_idx[j]]
    return X


def print_matrix(rev_doc_idx, rev_char_idx, X):
    print(",", end='')
    for i in rev_char_idx.keys():
        print(rev_char_idx[i], ", ", end='')
    print()
    for i in rev_doc_idx.keys():
        print(rev_doc_idx[i], ", ", end='')
        for j in rev_char_idx.keys():
            print(X[i, j], ", ", end='')
        print()

## The document by character matrix

### We create the file names

In [ ]:
np.set_printoptions(precision=4)

base = '../../corpus/Salammbo/'
english_chapters = ['salammbo_en_ch01.txt', 'salammbo_en_ch02.txt', 'salammbo_en_ch03.txt',
                    'salammbo_en_ch04.txt', 'salammbo_en_ch05.txt', 'salammbo_en_ch06.txt',
                    'salammbo_en_ch07.txt', 'salammbo_en_ch08.txt', 'salammbo_en_ch09.txt',
                    'salammbo_en_ch10.txt', 'salammbo_en_ch11.txt', 'salammbo_en_ch12.txt',
                    'salammbo_en_ch13.txt', 'salammbo_en_ch14.txt', 'salammbo_en_ch15.txt']
french_chapters = ['salammbo_ch01.txt', 'salammbo_ch02.txt', 'salammbo_ch03.txt',
                   'salammbo_ch04.txt', 'salammbo_ch05.txt', 'salammbo_ch06.txt',
                   'salammbo_ch07.txt', 'salammbo_ch08.txt', 'salammbo_ch09.txt',
                   'salammbo_ch10.txt', 'salammbo_ch11.txt', 'salammbo_ch12.txt',
                   'salammbo_ch13.txt', 'salammbo_ch14.txt', 'salammbo_ch15.txt']

french_labels = [str(i + 1) + '_fr' for i in range(15)]
english_labels = [str(i + 1) + '_en' for i in range(15)]
y_name = french_labels + english_labels
y_name

### The document indices

In [ ]:
# We build a doc x char matrix, where each document is represented in the character space.
y = french_chapters + english_chapters
ec = [base + file for file in english_chapters]
fc = [base + file for file in french_chapters]
# Document index and reverse index
doc_idx, rev_doc_idx = build_idx_from_list(y)
print(doc_idx)
print(rev_doc_idx)

### The counts

In [ ]:
counts_by_chapter = []
for file in fc + ec:
    text = open(file).read()
    counts = count_letters(text)
    counts_by_chapter.append(counts)
total_counts = merge_dicts(counts_by_chapter)
print(total_counts)

### The indices

In [ ]:
char_idx, rev_char_idx = build_idx_from_dict(total_counts)
# Character index and reverse index
print(char_idx)
print(rev_char_idx)

### The matrices

In [ ]:
X = build_matrix(doc_idx, rev_doc_idx, char_idx, rev_char_idx, counts_by_chapter)
print_matrix(rev_doc_idx, rev_char_idx, X)
print_matrix(rev_char_idx, rev_doc_idx, X.T)

## Scaling the matrices

In [ ]:
mean = np.mean(X[:, 0])
std = np.std(X[:, 0])
print('mean A:', mean, 'stdev A:', std)
print('Original:', X[0, 0], 'Standardized', (X[0, 0] - mean) / std)

# Boolean for normalization
normalize = True
# We scale the matrix
# This preprocessing combination seems to have the best results: normalize and standardize
if normalize:
    X_norm = Normalizer().fit_transform(X)
else:
    X_norm = X
X_scaled = StandardScaler().fit_transform(X_norm)

## Computation of the PCA with sklearn

In [ ]:
pca = PCA(n_components=3)
X_trunc = pca.fit_transform(X_scaled)
print(X_trunc)

## Visualizing the PCA

In [ ]:
plt.scatter(X_trunc[:, 0], X_trunc[:, 1], s=10)
for i in range(len(y_name)):
    # plt.plot(X_trunc[i, 0], X_trunc[i, 1], 'o')
    plt.annotate(y_name[i], (X_trunc[i, 0], X_trunc[i, 1]), fontsize=8)
plt.grid(True)
plt.show()
fig = plt.figure(1, figsize=(4, 3))
plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
y_color = [0] * 15 + [1] * 15
ax.scatter(X_trunc[:, 0], X_trunc[:, 1], X_trunc[:, 2], c=y_color,
           edgecolor='k', s=10)
for i in range(len(y_name)):
    ax.text(X_trunc[i, 0], X_trunc[i, 1], X_trunc[i, 2], y_name[i])
plt.show()

## Computation of the PCA with numpy

In [ ]:
U, s, Vt = np.linalg.svd(X_scaled, full_matrices=False)
print(np.shape(U), np.shape(Vt))
print('s:', s)
print(len(s))

cumulative_inertia = np.cumsum(s)
print(list(cumulative_inertia))

### Visualizing the Inertia

In [ ]:
plt.clf()
plt.plot(range(1, len(cumulative_inertia) + 1), cumulative_inertia, '-')
plt.show()

### The new coordinates

In [ ]:
Us = U @ np.diag(s)
# Axes may have opposed orientation. This is to have the same figure
I = np.identity(len(s))
I[0, 0] = -1
I[1, 1] = -1
Us = Us @ I
Us

In [ ]:
for i in range(len(y_name)):
    plt.plot(Us[i, 0], Us[i, 1], '*', markersize=8)
    plt.annotate(y_name[i], (Us[i, 0], Us[i, 1]), fontsize=8)
plt.show()
fig = plt.figure(1, figsize=(4, 3))
plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
y_color = [0] * 15 + [1] * 15
ax.scatter(Us[:, 0], Us[:, 1], Us[:, 2], c=y_color,
           edgecolor='k')
for i in range(len(y_name)):
    ax.text(Us[i, 0], Us[i, 1], Us[i, 2], y_name[i])
plt.show()

## The char-document matrix: Each character is represented in a space of documents

In [ ]:
yt = list(char_idx.keys())
print(yt)
X = X.T
if normalize:
    X_norm = Normalizer().fit_transform(X)
else:
    X_norm = X
X_scaled = StandardScaler().fit_transform(X_norm)
# X_std = StandardScaler().fit_transform(X)
U, s, Vt = np.linalg.svd(X_scaled, full_matrices=False)
Us = U @ np.diag(s)
print(Us)
plt.grid(True)
plt.axis('equal')
plt.scatter(Us[:, 0], Us[:, 1], s=3)
for i in range(len(yt)):
    #plt.plot(Us[i, 0], Us[i, 1], 'bo',markersize=5)
    plt.annotate(yt[i], (Us[i, 0], Us[i, 1]))
plt.show()

fig = plt.figure(1, figsize=(4, 3))
plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(Us[:, 0], Us[:, 1], Us[:, 2])
for i in range(len(yt)):
    ax.text(Us[i, 0], Us[i, 1], Us[i, 2], yt[i])
plt.show()